In [2]:
import capo 
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [28]:
#Get all the features in a condensed form.
d1 = np.loadtxt('stats_1e2.txt')
ants = d1[:,0]#antenna numbers
f1 = d1[:,-1]#standard devs of solved delays
m,v,f = capo.miriad.read_files(['zen.2456679.43314.xx.uvcRRE'],'auto','xx',verbose=True)
bad_antennae = [8,14,15,16,26,27,28,34,37,38,42,46,50,53,72,74,82,84,85,110]

feature_dict = {}
times = m['times']
for a,ff in zip(ants,f1):
    ant_flag = 1
    if a in bad_antennae: 
        print 'flagging: ',a 
        ant_flag = 0
    for samp,tsample in enumerate(times):
        if a not in feature_dict.keys():
            feature_dict[a] = {}            
    #feature_dict[a] = [np.hstack((ff,np.real(v[(a,a)]['xx'][time]),ant_flag))]
        feature_dict[a][samp] = [np.hstack((np.real(v[(a,a)]['xx'][samp]),ant_flag))]
    

   Reading zen.2456679.43314.xx.uvcRRE
flagging:  26.0
flagging:  27.0
flagging:  28.0
flagging:  8.0
flagging:  50.0
flagging:  53.0
flagging:  110.0
flagging:  82.0
flagging:  84.0
flagging:  85.0
flagging:  38.0
flagging:  37.0
flagging:  34.0
flagging:  15.0
flagging:  14.0
flagging:  16.0
flagging:  46.0
flagging:  42.0
flagging:  74.0
flagging:  72.0


[array([  0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   3.34624219,   4.25054312,
         5.81846094,   7.52479124,   9.73601723,  12.08374691,
        14.55045414,  16.82494736,  18.72192955,  20.17432785,
        21.28772926,  22.00339317,  22.57902336,  22.84867287,
        23.13844681,  23.17856979,  23.30441093,  23.21060181,
        23.13197517,  22.78497314,  22.54731369,  22.12143326,
        21.80599785,  21.43456459,  21.18190384,  20.91455269,
        20.80158806,  20.65184593,  20.62016678,  20.47453499,
        20.3763504 ,  20.14884377,  19.93280983,  19.60324287,
        19.28815269,  18.9113369 ,  18.59340858,  18.26516151,
        18.01861382,  17.7998867 ,  17.62939072,  17.44680023,
        17.30860329,  17.1494751 ,  16.92089272,  16.64335251,
        16.37208557,  16.07229614,  15.77547836,  15.5